In [1]:
# import functions
import idaes
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

from idaes.core import FlowsheetBlock
import idaes.logger as idaeslog
from pyomo.opt import TerminationCondition, SolverStatus
# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
# Import unit models from the model library
from idaes.generic_models.unit_models import Flash
# Import degrees of freedom tool
from idaes.core.util.model_statistics import degrees_of_freedom
# parmest (binary_param2)
from binary_param import linear
# import pressure and plot functions
from binary_fitting_functions import calc_outlet_xP
from binary_fitting_functions import plot_sens_analysis

In [2]:
# from HFC32_emimTf2N_PR import configuration
from HFC32_emimTf2N_PR import configuration

In [3]:
#need to make excel csv for 125
data = pd.read_csv('R32_emimTf2N.csv')
# print(data)

In [4]:
# 283
data_283= data[(data.temperature > 282) & (data.temperature< 284)]

# 298
data_298 = data[(data.temperature > 297) & (data.temperature< 299)]
data_298 = data_298.reset_index(drop=True)

# 323
data_323 = data[(data.temperature > 322) & (data.temperature< 324)]
data_323 = data_323.reset_index(drop=True)

# 323
data_348 = data[(data.temperature > 347) & (data.temperature< 349)]
data_348 = data_348.reset_index(drop=True)


In [5]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
m.fs.properties = GenericParameterBlock(default=configuration)
m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})
DOF_initial = degrees_of_freedom(m)
print('The initial DOF is {0}'.format(DOF_initial))

The initial DOF is 7


# Parameter estimation

In [6]:
print(data)

    temperature  pressure  x_R32  x_emimTf2N
0        283.15     10200  0.014       0.986
1        283.15    100100  0.136       0.864
2        283.15    250300  0.306       0.694
3        283.15    399300  0.448       0.552
4        283.15    549300  0.570       0.430
5        283.15    700400  0.672       0.328
6        283.15    849400  0.786       0.214
7        298.15      9600  0.010       0.990
8        298.15     99800  0.095       0.905
9        298.15    250200  0.218       0.782
10       298.15    399400  0.325       0.675
11       298.15    549400  0.417       0.583
12       298.15    699500  0.499       0.501
13       298.15    849500  0.570       0.430
14       298.15    999700  0.643       0.357
15       323.15     10200  0.005       0.995
16       323.15    100300  0.055       0.945
17       323.15    249700  0.133       0.867
18       323.15    400100  0.200       0.800
19       323.15    549500  0.264       0.736
20       323.15    700300  0.320       0.680
21       3

In [7]:
linear(data, configuration, 'R32', 'emimTf2N', "x_R32", "x_emimTf2N", 
    init_temp =  283.1, init_press =   399300 , init_x_c1 =    0.448, init_x_c2 = 0.552,
    init_kappa_2_1A = 1.2354, init_kappa_1_2A = 0.1695,
    init_kappa_2_1B = -1.387, init_kappa_1_2B = -0.2075, eps = 0.1, scaling_fac = 1e-9)

2021-08-17 12:43:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:43:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:43:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:43:40 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:43:42 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-08-17 12:43:44 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-08-17 12:43:45 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-17 12:43:47 [INFO] idaes.init.fs.state_block: Pro

  26  4.0855661e-01 3.95e-03 3.08e+14  -1.0 1.84e+05    -  1.00e+00 9.67e-04h 11
  27  9.3189190e-01 5.96e+00 7.69e+12  -1.0 1.72e+05    -  7.06e-01 9.90e-01w  1
In iteration 27, 31 Slacks too small, adjusting variable bounds
  28  1.3483898e-01 1.88e+00 3.78e+13  -1.0 1.13e+05    -  1.00e+00 9.90e-01w  1
  29  2.9213551e-01 3.80e+00 6.50e+00  -1.0 1.76e+04    -  1.00e+00 1.00e+00w  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  30  3.8081325e-01 5.47e-03 3.24e+14  -1.0 5.57e+03    -  7.06e-01 3.09e-02h  5
  31  3.7931256e-01 5.46e-03 3.24e+14  -1.0 1.51e+05    -  1.00e+00 1.95e-03h 10
  32  3.7790447e-01 5.46e-03 3.23e+14  -1.0 1.41e+05    -  8.20e-01 1.95e-03h 10
  33  3.7642819e-01 5.45e-03 3.23e+14  -1.0 1.49e+05    -  1.00e+00 1.95e-03h 10
  34  3.7503871e-01 5.45e-03 3.22e+14  -1.0 1.40e+05    -  8.27e-01 1.95e-03h 10
  35  3.7360128e-01 5.44e-03 3.21e+14  -1.0 1.46e+05    -  1.00e+00 1.95e-03h 10
  36  3.7224911e-01 5.44e-03 3.21e+14  -1.0 1